<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
response = requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup =BeautifulSoup(response.text, "html.parser")

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [26]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date andtime ()
</th>
<th scope="col"> 
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launchoutcome
</th>
<th scope="col">
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>4 June 2010,<br/>18:45
</td>
<td><a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">F9 v1.0</a><sup class="reference" id="cite_ref-MuskMay2012_13-0"><a href="#cite_note-MuskMay2012-13">[7]</a></sup><br/>B0003.1<sup class="reference" id="cite_ref-block_numbers_14-0"><a href="#cite_note-block_numbers-14">[8]</a></sup>
</td>
<td><a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">CCAFS</a>,<br/><a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Lau

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [12]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for th_element in first_launch_table.find_all('th'):
    name = extract_column_from_header(th_element)
    if name is not None and len(name) > 0:
        column_names.append(name)

Check the extracted column names


In [13]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [14]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [41]:
# Initialize a list to store the extracted launch data
launch_data = []

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # get table row
    for rows in table.find_all("tr"):
        # check to see if first table heading is a number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # get table element
        row = rows.find_all('td')
        
        # if it is a number, save cells in a dictionary
        if flag:
            launch_dict = {}

            # Flight Number value
            launch_dict["Flight No."] = flight_number
            
            # Date and Time value
            date_time = row[0].text.strip()
            date_time_list = date_time.split(',')
            if len(date_time_list) == 2:
                date = date_time_list[0].strip()
                time = date_time_list[1].strip()
                launch_dict["Date"] = date
                launch_dict["Time"] = time
            
            # Booster version
            bv = row[1].text.strip()
            launch_dict["Version Booster"] = bv
            
            # Launch Site
            launch_site = row[2].text.strip()
            launch_dict["Launch Site"] = launch_site
            
            # Payload
            payload = row[3].text.strip()
            launch_dict["Payload"] = payload
            
            # Payload Mass
            payload_mass = row[4].text.strip()
            launch_dict["Payload mass"] = payload_mass
            
            # Orbit
            orbit = row[5].text.strip()
            launch_dict["Orbit"] = orbit
            
            # Customer
            customer = row[6].text.strip()
            launch_dict["Customer"] = customer
            
            # Launch outcome
            launch_outcome = row[7].text.strip()
            launch_dict["Launch outcome"] = launch_outcome
            
            # Booster landing
            booster_landing = row[8].text.strip()
            launch_dict["Booster landing"] = booster_landing

            # Append the launch data to the list
            launch_data.append(launch_dict)

# Print the extracted launch data
for launch in launch_data:
    print(launch)


{'Flight No.': '1', 'Date': '4 June 2010', 'Time': '18:45', 'Version Booster': 'F9 v1.0[7]B0003.1[8]', 'Launch Site': 'CCAFS,SLC-40', 'Payload': 'Dragon Spacecraft Qualification Unit', 'Payload mass': '', 'Orbit': 'LEO', 'Customer': 'SpaceX', 'Launch outcome': 'Success', 'Booster landing': 'Failure[9][10](parachute)'}
{'Flight No.': '2', 'Date': '8 December 2010', 'Time': '15:43[13]', 'Version Booster': 'F9 v1.0[7]B0004.1[8]', 'Launch Site': 'CCAFS,SLC-40', 'Payload': 'Dragon demo flight C1(Dragon C101)', 'Payload mass': '', 'Orbit': 'LEO (ISS)', 'Customer': 'NASA (COTS)\nNRO', 'Launch outcome': 'Success[9]', 'Booster landing': 'Failure[9][14](parachute)'}
{'Flight No.': '3', 'Date': '22 May 2012', 'Time': '07:44[17]', 'Version Booster': 'F9 v1.0[7]B0005.1[8]', 'Launch Site': 'CCAFS,SLC-40', 'Payload': 'Dragon demo flight C2+[18](Dragon C102)', 'Payload mass': '525\xa0kg (1,157\xa0lb)[19]', 'Orbit': 'LEO (ISS)', 'Customer': 'NASA (COTS)', 'Launch outcome': 'Success[20]', 'Booster landi

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [35]:
df= pd.DataFrame(launch_data)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [42]:
df.to_csv('spacex_web_scraped.csv', index=False)
df

,Flight No.,Date,Time,Version Booster,Launch Site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010,18:45,F9 v1.0[7]B0003.1[8],"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,,LEO,SpaceX,Success,Failure[9][10](parachute)
1,2,8 December 2010,15:43[13],F9 v1.0[7]B0004.1[8],"CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),,LEO (ISS),NASA (COTS)\nNRO,Success[9],Failure[9][14](parachute)
2,3,22 May 2012,07:44[17],F9 v1.0[7]B0005.1[8],"CCAFS,SLC-40",Dragon demo flight C2+[18](Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],No attempt
3,4,8 October 2012,00:35[21],F9 v1.0[7]B0006.1[8],"CCAFS,SLC-40",SpaceX CRS-1[22](Dragon C103),"4,700 kg (10,400 lb)",LEO (ISS),NASA (CRS),Success,No attempt
4,5,1 March 2013,15:10,F9 v1.0[7]B0007.1[8],"CCAFS,SLC-40",SpaceX CRS-2[22](Dragon C104),"4,877 kg (10,752 lb)",LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...,...
116,117,NaN,NaN,F9 B5 ♺B1051.10[657],"CCSFS,SLC-40",Starlink 27 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship)
117,118,NaN,NaN,F9 B5 ♺B1058.8[660],"KSC,LC-39A[661]",Starlink 26 v1.0 (52 Satellites) Capella-6 &Ty...,"~14,000 kg (31,000 lb)",LEO,SpaceX Capella Space and Tyvak,Success,Success(drone ship)
118,119,NaN,NaN,F9 B5 ♺B1063.2[665],"CCSFS,SLC-40[665]",Starlink 28 v1.0 (60 Satellites)[665],"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship)
119,120,NaN,NaN,F9 B5 B1067.1[668],"KSC,LC-39A",SpaceX CRS-22(Dragon C209.1),"3,328 kg (7,337 lb)",LEO (ISS),NASA (CRS),Success,Success(drone ship)


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
